In [19]:
from FINData import FINData
from vectorbtpro import vbt
from numba import njit

data=FINData.pull(
    ['HPG'],
    start='2014-01-01',
    end=None
)

In [20]:
import pandas as pd 
import numpy as np
def get_med_price(high, low):
    return (high + low) / 2

def get_atr(high, low, close, period):
    tr0 = abs(high - low)
    tr1 = abs(high - close.shift())
    tr2 = abs(low - close.shift())
    tr = pd.concat((tr0, tr1, tr2), axis=1).max(axis=1)  
    atr = tr.ewm(
        alpha=1 / period, 
        adjust=False, 
        min_periods=period).mean()  
    return atr

def get_basic_bands(med_price, atr, multiplier):
    matr = multiplier * atr
    upper = med_price + matr
    lower = med_price - matr
    return upper, lower

def get_final_bands(close, upper, lower):  
    trend = pd.Series(np.full(close.shape, np.nan), index=close.index)
    dir_ = pd.Series(np.full(close.shape, 1), index=close.index)
    long = pd.Series(np.full(close.shape, np.nan), index=close.index)
    short = pd.Series(np.full(close.shape, np.nan), index=close.index)

    for i in range(1, close.shape[0]):  
        if close.iloc[i] > upper.iloc[i - 1]:
            dir_.iloc[i] = 1
        elif close.iloc[i] < lower.iloc[i - 1]:
            dir_.iloc[i] = -1
        else:
            dir_.iloc[i] = dir_.iloc[i - 1]
            if dir_.iloc[i] > 0 and lower.iloc[i] < lower.iloc[i - 1]:
                lower.iloc[i] = lower.iloc[i - 1]
            if dir_.iloc[i] < 0 and upper.iloc[i] > upper.iloc[i - 1]:
                upper.iloc[i] = upper.iloc[i - 1]

        if dir_.iloc[i] > 0:
            trend.iloc[i] = long.iloc[i] = lower.iloc[i]
        else:
            trend.iloc[i] = short.iloc[i] = upper.iloc[i]
            
    return trend, dir_, long, short

def supertrend(high, low, close, period=7, multiplier=3):
    med_price = get_med_price(high, low)
    atr = get_atr(high, low, close, period)
    upper, lower = get_basic_bands(med_price, atr, multiplier)
    return get_final_bands(close, upper, lower)

In [21]:
high = data.get('High')
low = data.get('Low')
close = data.get('Close')

supert, superd, superl, supers = supertrend(
    high['HPG'], 
    low['HPG'], 
    close['HPG']
)
supert

DateTime
2014-01-02 00:00:00+00:00          NaN
2014-01-03 00:00:00+00:00          NaN
2014-01-06 00:00:00+00:00          NaN
2014-01-07 00:00:00+00:00          NaN
2014-01-08 00:00:00+00:00          NaN
                               ...    
2025-11-17 00:00:00+00:00    28.199854
2025-11-18 00:00:00+00:00    28.199854
2025-11-19 00:00:00+00:00    28.199854
2025-11-20 00:00:00+00:00    28.199854
2025-11-21 00:00:00+00:00    28.199854
Length: 2965, dtype: float64

In [22]:
date_range = slice('2014-01-01', '2025-11-21')
fig = close.loc[date_range, 'HPG'].rename('Close').vbt.plot()  
supers.loc[date_range].rename('Short').vbt.plot(fig=fig)
superl.loc[date_range].rename('Long').vbt.plot(fig=fig)

fig.show()

In [23]:
def get_atr_np(high, low, close, period):
    shifted_close = vbt.nb.fshift_1d_nb(close)  
    tr0 = np.abs(high - low)
    tr1 = np.abs(high - shifted_close)
    tr2 = np.abs(low - shifted_close)
    tr = np.column_stack((tr0, tr1, tr2)).max(axis=1)  
    atr = vbt.nb.wwm_mean_1d_nb(tr, period)  
    return atr

In [24]:
@njit
def get_final_bands_nb(close, upper, lower):  
    trend = np.full(close.shape, np.nan)  
    dir_ = np.full(close.shape, 1)
    long = np.full(close.shape, np.nan)
    short = np.full(close.shape, np.nan)

    for i in range(1, close.shape[0]):
        if close[i] > upper[i - 1]:  
            dir_[i] = 1
        elif close[i] < lower[i - 1]:
            dir_[i] = -1
        else:
            dir_[i] = dir_[i - 1]
            if dir_[i] > 0 and lower[i] < lower[i - 1]:
                lower[i] = lower[i - 1]
            if dir_[i] < 0 and upper[i] > upper[i - 1]:
                upper[i] = upper[i - 1]

        if dir_[i] > 0:
            trend[i] = long[i] = lower[i]
        else:
            trend[i] = short[i] = upper[i]
            
    return trend, dir_, long, short

In [25]:
def faster_supertrend(high, low, close, period=7, multiplier=3):
    med_price = get_med_price(high, low)
    atr = get_atr_np(high, low, close, period)
    upper, lower = get_basic_bands(med_price, atr, multiplier)
    return get_final_bands_nb(close, upper, lower)

In [27]:
supert, superd, superl, supers = faster_supertrend(
    high['HPG'].values,  
    low['HPG'].values, 
    close['HPG'].values
)

In [29]:
pd.Series(supert, index=close.index)

DateTime
2014-01-02 00:00:00+00:00          NaN
2014-01-03 00:00:00+00:00          NaN
2014-01-06 00:00:00+00:00          NaN
2014-01-07 00:00:00+00:00          NaN
2014-01-08 00:00:00+00:00          NaN
                               ...    
2025-11-17 00:00:00+00:00    28.199854
2025-11-18 00:00:00+00:00    28.199854
2025-11-19 00:00:00+00:00    28.199854
2025-11-20 00:00:00+00:00    28.199854
2025-11-21 00:00:00+00:00    28.199854
Length: 2965, dtype: float64

In [30]:
import talib

def faster_supertrend_talib(high, low, close, period=7, multiplier=3):
    avg_price = talib.MEDPRICE(high, low)  
    atr = talib.ATR(high, low, close, period)
    upper, lower = get_basic_bands(avg_price, atr, multiplier)
    return get_final_bands_nb(close, upper, lower)

faster_supertrend_talib(
    high['HPG'].values, 
    low['HPG'].values, 
    close['HPG'].values
)

(array([        nan,         nan,         nan, ..., 28.19985395,
        28.19985395, 28.19985395]),
 array([ 1,  1,  1, ..., -1, -1, -1], dtype=int64),
 array([nan, nan, nan, ..., nan, nan, nan]),
 array([        nan,         nan,         nan, ..., 28.19985395,
        28.19985395, 28.19985395]))

In [32]:
SuperTrend = vbt.IF(
    class_name='SuperTrend',
    short_name='st',
    input_names=['high', 'low', 'close'],
    param_names=['period', 'multiplier'],
    output_names=['supert', 'superd', 'superl', 'supers']
).with_apply_func(
    faster_supertrend_talib, 
    takes_1d=True,  
    period=7,  
    multiplier=3
)

In [41]:
st = SuperTrend.run(high, low, close)
st.supert

symbol,HPG
DateTime,
2014-01-02 00:00:00+00:00,NaN
2014-01-03 00:00:00+00:00,NaN
2014-01-06 00:00:00+00:00,NaN
2014-01-07 00:00:00+00:00,NaN
2014-01-08 00:00:00+00:00,NaN
...,...
2025-11-17 00:00:00+00:00,28.199854
2025-11-18 00:00:00+00:00,28.199854
2025-11-19 00:00:00+00:00,28.199854


In [35]:
class SuperTrend(SuperTrend):
    def plot(self, 
             column=None,  
             close_kwargs=None,  
             superl_kwargs=None,
             supers_kwargs=None,
             fig=None,  
             **layout_kwargs):  
        close_kwargs = close_kwargs if close_kwargs else {}
        superl_kwargs = superl_kwargs if superl_kwargs else {}
        supers_kwargs = supers_kwargs if supers_kwargs else {}
        
        close = self.select_col_from_obj(self.close, column).rename('Close')
        supers = self.select_col_from_obj(self.supers, column).rename('Short')
        superl = self.select_col_from_obj(self.superl, column).rename('Long')
        
        fig = close.vbt.plot(fig=fig, **close_kwargs, **layout_kwargs)  
        supers.vbt.plot(fig=fig, **supers_kwargs)
        superl.vbt.plot(fig=fig, **superl_kwargs)
        
        return fig

In [37]:
st = SuperTrend.run(high, low, close)
st.loc[date_range, 'HPG'].plot(
    superl_kwargs=dict(trace_kwargs=dict(line_color='limegreen')),
    supers_kwargs=dict(trace_kwargs=dict(line_color='red'))
).show()

In [39]:
entries = (~st.superl.isnull()).vbt.signals.fshift()  
exits = (~st.supers.isnull()).vbt.signals.fshift()

In [54]:
pf = vbt.Portfolio.from_signals(
    close=close, 
    entries=entries, 
    exits=exits, 
    fees=0.001, 
    freq='1d'
)

In [55]:
pf['HPG'].stats()

Start Index                   2014-01-02 00:00:00+00:00
End Index                     2025-11-21 00:00:00+00:00
Total Duration                       2965 days 00:00:00
Start Value                                       100.0
Min Value                                     87.438273
Max Value                                    1559.87456
End Value                                   1376.040644
Total Return [%]                            1276.040644
Benchmark Return [%]                        1112.389381
Position Coverage [%]                         60.539629
Max Gross Exposure [%]                            100.0
Max Drawdown [%]                              27.209257
Max Drawdown Duration                 561 days 00:00:00
Total Orders                                         80
Total Fees Paid                               40.417162
Total Trades                                         40
Win Rate [%]                                       50.0
Best Trade [%]                                 9

In [47]:
periods = np.arange(4, 20)
multipliers = np.arange(20, 41) / 10  

st = SuperTrend.run(
    high, low, close, 
    period=periods, 
    multiplier=multipliers,
    param_product=True
)

In [48]:
st.wrapper.columns

MultiIndex([( 4, 2.0, 'HPG'),
            ( 4, 2.1, 'HPG'),
            ( 4, 2.2, 'HPG'),
            ( 4, 2.3, 'HPG'),
            ( 4, 2.4, 'HPG'),
            ( 4, 2.5, 'HPG'),
            ( 4, 2.6, 'HPG'),
            ( 4, 2.7, 'HPG'),
            ( 4, 2.8, 'HPG'),
            ( 4, 2.9, 'HPG'),
            ...
            (19, 3.1, 'HPG'),
            (19, 3.2, 'HPG'),
            (19, 3.3, 'HPG'),
            (19, 3.4, 'HPG'),
            (19, 3.5, 'HPG'),
            (19, 3.6, 'HPG'),
            (19, 3.7, 'HPG'),
            (19, 3.8, 'HPG'),
            (19, 3.9, 'HPG'),
            (19, 4.0, 'HPG')],
           names=['st_period', 'st_multiplier', 'symbol'], length=336)

In [49]:
print(st.getsize())

32.0 MB


d:\stockvip\vectorbtLearn\venv\Lib\site-packages\vectorbtpro\utils\module_.py:354: VBTWarning: Please install dill. See https://pypi.org/project/dill/.


In [57]:
entries = (~st.superl.isnull()).vbt.signals.fshift()
exits = (~st.supers.isnull()).vbt.signals.fshift()

pf = vbt.Portfolio.from_signals(
    close=close, 
    entries=entries, 
    exits=exits, 
    fees=0.001, 
    freq='1d'
)

In [58]:
pf.sharpe_ratio.vbt.heatmap(
    x_level='st_period', 
    y_level='st_multiplier',
    slider_level='symbol'
)

In [60]:
stats_df = pf.stats([
    'end_value',
    'total_return', 
    'total_trades', 
    'win_rate', 
    'expectancy'
], agg_func=None)  
stats_df.sort_values("Total Return [%]",ascending=False)

End Value  Total Return [%]  Total Trades  \
st_period st_multiplier symbol                                                
6         2.3           HPG     2271.047091       2171.047091            52   
7         2.3           HPG     2167.534562       2067.534562            52   
15        2.8           HPG     2063.283334       1963.283334            41   
16        2.8           HPG     1981.044960       1881.044960            42   
          2.9           HPG     1971.853077       1871.853077            41   
...                                     ...               ...           ...   
12        2.1           HPG      654.618605        554.618605            67   
10        2.0           HPG      612.588578        512.588578            69   
11        2.0           HPG      606.345418        506.345418            69   
12        2.0           HPG      591.645773        491.645773            69   
13        2.0           HPG      569.349656        469.349656            71   

                                Win Rate [%]  Expectancy  
st_period st_multiplier symbol                            
6         2.3           HPG        57.692308   41.750906  
7         2.3           HPG        57.692308   39.760280  
15        2.8           HPG        51.219512   47.884959  
16        2.8           HPG        52.380952   44.786785  
          2.9           HPG        51.219512   45.654953  
...                                      ...         ...  
12        2.1           HPG        46.268657    8.277890  
10        2.0           HPG        42.028986    7.428820  
11        2.0           HPG        40.579710    7.338339  
12        2.0           HPG        40.579710    7.125301  
13        2.0           HPG        39.436620    6.610559  

[336 rows x 5 columns]

In [ ]:
clean_entries, clean_exits =entries.vbt.signals.clean(exits)
clean_entries

st_period                     4                                             \
st_multiplier                2.0    2.1    2.2    2.3    2.4    2.5    2.6   
symbol                       HPG    HPG    HPG    HPG    HPG    HPG    HPG   
DateTime                                                                     
2014-01-02 00:00:00+00:00  False  False  False  False  False  False  False   
2014-01-03 00:00:00+00:00  False  False  False  False  False  False  False   
2014-01-06 00:00:00+00:00  False  False  False  False  False  False  False   
2014-01-07 00:00:00+00:00  False  False  False  False  False  False  False   
2014-01-08 00:00:00+00:00  False  False  False  False  False  False  False   
...                          ...    ...    ...    ...    ...    ...    ...   
2025-11-17 00:00:00+00:00  False  False  False  False  False  False  False   
2025-11-18 00:00:00+00:00  False  False  False  False  False  False  False   
2025-11-19 00:00:00+00:00   True  False  False  False  False  False  False   
2025-11-20 00:00:00+00:00  False   True   True  False  False  False  False   
2025-11-21 00:00:00+00:00  False  False  False  False  False  False  False   

st_period                                       ...     19                \
st_multiplier                2.7    2.8    2.9  ...    3.1    3.2    3.3   
symbol                       HPG    HPG    HPG  ...    HPG    HPG    HPG   
DateTime                                        ...                        
2014-01-02 00:00:00+00:00  False  False  False  ...  False  False  False   
2014-01-03 00:00:00+00:00  False  False  False  ...  False  False  False   
2014-01-06 00:00:00+00:00  False  False  False  ...  False  False  False   
2014-01-07 00:00:00+00:00  False  False  False  ...  False  False  False   
2014-01-08 00:00:00+00:00  False  False  False  ...  False  False  False   
...                          ...    ...    ...  ...    ...    ...    ...   
2025-11-17 00:00:00+00:00  False  False  False  ...  False  False  False   
2025-11-18 00:00:00+00:00  False  False  False  ...  False  False  False   
2025-11-19 00:00:00+00:00  False  False  False  ...  False  False  False   
2025-11-20 00:00:00+00:00  False  False  False  ...  False  False  False   
2025-11-21 00:00:00+00:00  False  False  False  ...  False  False  False   

st_period                                                                   
st_multiplier                3.4    3.5    3.6    3.7    3.8    3.9    4.0  
symbol                       HPG    HPG    HPG    HPG    HPG    HPG    HPG  
DateTime                                                                    
2014-01-02 00:00:00+00:00  False  False  False  False  False  False  False  
2014-01-03 00:00:00+00:00  False  False  False  False  False  False  False  
2014-01-06 00:00:00+00:00  False  False  False  False  False  False  False  
2014-01-07 00:00:00+00:00  False  False  False  False  False  False  False  
2014-01-08 00:00:00+00:00  False  False  False  False  False  False  False  
...                          ...    ...    ...    ...    ...    ...    ...  
2025-11-17 00:00:00+00:00  False  False  False  False  False  False  False  
2025-11-18 00:00:00+00:00  False  False  False  False  False  False  False  
2025-11-19 00:00:00+00:00  False  False  False  False  False  False  False  
2025-11-20 00:00:00+00:00  False  False  False  False  False  False  False  
2025-11-21 00:00:00+00:00  False  False  False  False  False  False  False  

[2965 rows x 336 columns]

In [64]:
pf[(6, 2.3, 'HPG')].plot(settings=dict(bm_returns=False)).show()